In [ ]:
print("hello World")

hello World


Make sure that we are on the right conda environment (ms2lda_env)

In [2]:
import sys
print(sys.executable)

/home/ioannis/miniconda3/envs/ms2lda_env/bin/python


In [3]:
from pyteomics import mgf

In [5]:
mgf_file = "datasets/MSnLib/20241003_mcedrug_pos_msn.mgf"
spectra = mgf.read(mgf_file)

/home/ioannis/miniconda3/envs/ms2lda_env/lib/python3.11/site-packages/pyteomics/mgf.py:388: UserWarning: IndexedMGF object has an empty index for file datasets/MSnLib/20241003_mcedrug_pos_msn.mgf. If this is unexpected, consider adjusting `label` or setting `index_by_scans=True`.
  warnings.warn(text)


Since the MSnLib doesn't use the common TITLE for each spectra, it can't build an index for each spectra, so:

In [6]:
spectra = mgf.MGF(mgf_file)

In [8]:
first = next(spectra)
print(first['params'])

{'name': 'propranolol', 'description': 'MCEDRUG', 'exactmass': '259.15723', 'formula': 'C16H21NO2', 'inchi': 'InChI=1S/C16H21NO2/c1-12(2)17-10-14(18)11-19-16-9-5-7-13-6-3-4-8-15(13)16/h3-9,12,14,17-18H,10-11H2,1-2H3', 'inchiaux': 'AQHHHDLHHXJYJD-UHFFFAOYSA-N', 'smiles': 'CC(C)NCC(O)COc1cccc2ccccc12', 'feature_id': '20240408_pluskal_fda_drugs_5005_G14_id_MSn_positive.mzML msn trees:5', 'mslevel': '2', 'rtinseconds': 46.26, 'adduct': '[M+H]+', 'pepmass': (260.1645, None), 'charge': [1], 'feature_ms1_height': '1.485E8', 'spectype': 'ALL_ENERGIES', 'collision_energy': '[20.0, 15.0, 60.0, 30.0]', 'fragmentation_method': 'HCD', 'isolation_window': '1.2000000476839432', 'acquisition': 'Commercial', 'instrument_type': 'Orbitrap', 'source_instrument': 'Orbitrap ID-X', 'ims_type': 'none', 'ion_source': 'ESI', 'ionmode': 'Positive', 'pi': 'Tomas Pluskal', 'datacollector': 'Corinna Brungs', 'dataset_id': 'MSV000094528', 'usi': '[mzspec:MSV000094528:20240408_pluskal_fda_drugs_5005_G14_id_MSn_positi

Each spectra has these parameters:

In [9]:
print(list(first['params'].keys()))

['name', 'description', 'exactmass', 'formula', 'inchi', 'inchiaux', 'smiles', 'feature_id', 'mslevel', 'rtinseconds', 'adduct', 'pepmass', 'charge', 'feature_ms1_height', 'spectype', 'collision_energy', 'fragmentation_method', 'isolation_window', 'acquisition', 'instrument_type', 'source_instrument', 'ims_type', 'ion_source', 'ionmode', 'pi', 'datacollector', 'dataset_id', 'usi', 'scans', 'precursor_purity', 'quality_chimeric', 'quality_explained_intensity', 'quality_explained_signals', 'num peaks']


How many spectra are in this mfg file? :

In [10]:
count = sum(1 for _ in mgf.MGF(mgf_file))
print("Total spectra in file:", count)

Total spectra in file: 123389


In [12]:
%%time
count = 0
for spectrum in mgf.MGF(mgf_file):
    if spectrum['params'].get('spectype') == 'ALL_MSN_TO_PSEUDO_MS2':
        count += 1

print("Spectra with SPECTYPE=ALL_MSN_TO_PSEUDO_MS2:", count)

Spectra with SPECTYPE=ALL_MSN_TO_PSEUDO_MS2: 2180
CPU times: user 14.6 s, sys: 360 ms, total: 15 s
Wall time: 26.1 s


Now let's explore the ms2 mfg file, which should have the same number of pseudo_ms2 plus the real MS2

In [37]:
mgf_ms2 = 'datasets/MSnLib/20241003_mcedrug_pos_ms2.mgf'

In [38]:
spectra2 = mgf.MGF(mgf_ms2)
first2 = next(spectra2)
print(list(first2['params'].keys()))

['name', 'description', 'exactmass', 'formula', 'inchi', 'inchiaux', 'smiles', 'feature_id', 'mslevel', 'rtinseconds', 'adduct', 'pepmass', 'charge', 'feature_ms1_height', 'spectype', 'collision_energy', 'fragmentation_method', 'isolation_window', 'acquisition', 'instrument_type', 'source_instrument', 'ims_type', 'ion_source', 'ionmode', 'pi', 'datacollector', 'dataset_id', 'usi', 'scans', 'precursor_purity', 'quality_chimeric', 'quality_explained_intensity', 'quality_explained_signals', 'num peaks']


In [39]:
%%time
spectra2 = mgf.MGF(mgf_ms2)
count2 = sum(1 for _ in spectra2)
print("Total spectra in file:", count2)

Total spectra in file: 15268
CPU times: user 1.53 s, sys: 33.5 ms, total: 1.57 s
Wall time: 2.01 s


In [40]:
%%time
spectra2 = mgf.MGF(mgf_ms2)
count = 0
for spectrum in spectra2:
    if spectrum['params'].get('spectype') == 'ALL_MSN_TO_PSEUDO_MS2':
        count += 1
print("Spectra with SPECTYPE=ALL_MSN_TO_PSEUDO_MS2:", count)

Spectra with SPECTYPE=ALL_MSN_TO_PSEUDO_MS2: 2180
CPU times: user 1.43 s, sys: 42.5 ms, total: 1.47 s
Wall time: 1.86 s


So we have 2180 pseudo_ms2 in both files

In [42]:
%%time
inp = "datasets/MSnLib/20241003_mcedrug_pos_ms2.mgf"
out = "datasets/MSnLib/filtered_pseudoMS2.mgf"

# Collect only pseudo-MS2 spectra
filtered = []
for spectrum in mgf.MGF(inp):
    if spectrum['params'].get('spectype') == 'ALL_MSN_TO_PSEUDO_MS2':
        filtered.append(spectrum)

print("Kept spectra:", len(filtered))

# Write them back into a new MGF file
with mgf.write(filtered, out) as writer:
    pass

print("Filtered MGF written to:", out)


Kept spectra: 2180
Filtered MGF written to: datasets/MSnLib/filtered_pseudoMS2.mgf
CPU times: user 5.03 s, sys: 202 ms, total: 5.24 s
Wall time: 6.04 s


In [64]:
%cd /home/ioannis/thesis_metabolomics/tools/MS2LDA/

/mnt/c/Users/user/Documents/thesis_metabolomics/tools/MS2LDA


In [65]:
!pwd

/mnt/c/Users/user/Documents/thesis_metabolomics/tools/MS2LDA


In [62]:
!ls /home/ioannis/thesis_metabolomics/tools/MS2LDA/MS2LDA/Add_On/Spec2Vec/model_positive_mode

150225_CleanedLibraries_Spec2Vec_pos_embeddings.npy
150225_CombLibraries_spectra.db
150225_Spec2Vec_pos_CleanedLibraries.model
150225_Spec2Vec_pos_CleanedLibraries.model.syn1neg.npy
150225_Spec2Vec_pos_CleanedLibraries.model.wv.vectors.npy


In [66]:
%%time
!bash ./run_analysis.sh \
  --dataset /home/ioannis/thesis_metabolomics/datasets/MSnLib/filtered_pseudoMS2.mgf \
  --n-motifs 200 \
  --n-iterations 1000 \
  --output-folder /home/ioannis/thesis_metabolomics/datasets/results

PYTHONPATH set to: /mnt/c/Users/user/Documents/thesis_metabolomics/tools/MS2LDA:
Running MS2LDA CLI script...

--- Running MS2LDA with parameters ---
{
  "dataset_parameters": {
    "acquisition_type": "DDA",
    "charge": 1,
    "significant_digits": 2,
    "name": "ms2lda_filtered_pseudoMS2",
    "output_folder": "/home/ioannis/thesis_metabolomics/datasets/results"
  },
  "train_parameters": {
    "parallel": 3,
    "workers": 0
  },
  "model_parameters": {
    "rm_top": 0,
    "min_cf": 0,
    "min_df": 3,
    "alpha": 0.6,
    "eta": 0.01,
    "seed": 42
  },
  "convergence_parameters": {
    "step_size": 50,
    "window_size": 10,
    "threshold": 0.005,
    "type": "perplexity_history"
  },
  "annotation_parameters": {
    "criterium": "biggest",
    "cosine_similarity": 0.9,
    "n_mols_retrieved": 5,
    "s2v_model_path": "../MS2LDA/MS2LDA/Add_On/Spec2Vec/model_positive_mode/150225_Spec2Vec_pos_CleanedLibraries.model",
    "s2v_library_embeddings": "../MS2LDA/MS2LDA/Add_On/Spec